In [31]:
# !pip install twarc

In [148]:
import pandas as pd
import json
import numpy as np
from ast import literal_eval

In [128]:
tweet_fp = '../data/raw/2020-10-02_clean-dataset.tsv'
to_fp = '../data/temp/sample_id.txt'
seed = 400
sample_size = 10000
rehydrated_fp = '../data/temp/tweets_hydrated.jsonl'
rehydrated_df_fp = '../data/temp/tweets_processing.csv'

In [67]:
df = pd.read_table(tweet_fp)
print(df.shape)
df.head()

(917217, 5)


,tweet_id,date,time,lang,country_code
0,1311878600208134149,2020-10-02,04:00:06,en,NaN
1,1311878600518438912,2020-10-02,04:00:06,es,NaN
2,1311878600652730372,2020-10-02,04:00:06,und,DE
3,1311878600682070023,2020-10-02,04:00:06,or,NaN
4,1311878601046921217,2020-10-02,04:00:06,en,NaN


In [68]:
id_sample = df['tweet_id'].sample(n=sample_size, replace = False, random_state = seed)
id_sample.to_csv(to_fp, index=False, header=False)

Extract from jsonL and parses hashtag into dataframe

In [135]:
with open(rehydrated_fp, encoding = 'utf-16') as rh_fh:
    json_content = rh_fh.read()
rows = []
for line in json_content.splitlines():
    rows.append(json.loads(line))
tweets = pd.DataFrame(rows)
tweets['hashtags'] = tweets['entities'].apply(lambda x: [tag['text'] for tag in x['hashtags']])
tweets.to_csv(rehydrated_df_fp, index = False)

In [186]:
tweets = pd.read_csv(rehydrated_df_fp)
tweets['hashtags'] = tweets['hashtags'].apply(literal_eval)

In [187]:
tweets.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quoted_status_permalink,quoted_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,extended_entities,hashtags
0,Fri Oct 02 04:25:52 +0000 2020,1311885083226169344,1311885083226169344,#TheTrumpDepression #BidenIsWinning\n#JoeBiden...,False,"[0, 278]","{'hashtags': [{'text': 'TheTrumpDepression', '...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,"{'url': 'https://t.co/K3uEqIKDvf', 'expanded':...",{'created_at': 'Fri Oct 02 01:12:32 +0000 2020...,0,0,False,False,False,und,NaN,"[TheTrumpDepression, BidenIsWinning, JoeBiden,..."
1,Fri Oct 02 04:44:14 +0000 2020,1311889706661994496,1311889706661994496,@Roscommon_Cat What's the bet we won't get his...,False,"[15, 294]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.311881e+18,1.311881e+18,...,NaN,NaN,0,2,False,False,NaN,en,NaN,[]
2,Fri Oct 02 04:59:03 +0000 2020,1311893436547715072,1311893436547715072,HAAHAJSKDLEKDLELDL https://t.co/l9OZSTbrrF,False,"[0, 18]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,"{'url': 'https://t.co/l9OZSTbrrF', 'expanded':...",{'created_at': 'Fri Oct 02 04:56:04 +0000 2020...,0,0,False,False,False,fi,NaN,[]
3,Fri Oct 02 05:35:50 +0000 2020,1311902690381881344,1311902690381881344,#Trump just tested positive for #COVID19\n\nht...,False,"[0, 65]","{'hashtags': [{'text': 'Trump', 'indices': [0,...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,NaN,NaN,0,0,False,False,False,en,NaN,"[Trump, COVID19]"
4,Fri Oct 02 05:45:16 +0000 2020,1311905064659771393,1311905064659771393,woj in shambles after this one https://t.co/GV...,False,"[0, 30]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,"{'url': 'https://t.co/GVmt9337To', 'expanded':...",{'created_at': 'Fri Oct 02 04:54:31 +0000 2020...,0,1,False,False,False,en,NaN,[]


In [227]:
# number of tweets that involves Trump
tweets.loc[tweets['full_text'].apply(lambda x: x.lower()).str.contains('trump')].shape

(2719, 31)

---------------------------------

### Find fake news hashtags
Problem: they are more like dumb tweets rather than misinformation

In [228]:
def filter_tweet_text(tweets, keyword, case = False):
    if case:
        tweets.loc[tweets['full_text'].str.contains(keyword)]
    return tweets.loc[tweets['full_text'].str.lower().str.contains(keyword.lower())]

In [199]:
def get_hashtag_counts(tweets, search_type = 'list'):
    return pd.Series(tweets['hashtags'].sum()).value_counts()

In [204]:
def filter_hashtags(hashtags, keyword, case = False):
    if not case:
        return [x for x in hashtag_counts.index[:1000] if keyword.lower() in x.lower()]
    else:
        return [x for x in hashtag_counts.index[:1000] if keyword in x]

In [205]:
def find_tweet_from_hashtag(tweets, tag):
    return tweets.loc[tweets['hashtags'].apply(lambda x: tag in x)]

In [216]:
def filter_top_hashtags(hashtag_counts, keyword, k = 10, case = False):
    keys = filter_hashtags(hashtag_counts.index, keyword, case=case)
    return hashtag_counts[keys].sort_values(ascending = False).iloc[:k]

In [207]:
hashtag_counts = get_hashtag_counts(tweets)
hashtag_counts.head()

COVID19          1105
coronavirus       276
TrumpHasCovid     268
Trump             182
Covid19           120
dtype: int64

In [208]:
filter_top_hashtags(hashtag_counts, 'hoax')

Hoax               4
hoax               4
coronavirushoax    2
NOHOAX             1
JustAHoax          1
virushoax          1
trumpisthehoax     1
dtype: int64

In [209]:
filter_top_hashtags(hashtag_counts, 'democrat', k = 20)

Democrats                         4
Democrat                          3
WalkAwayFromDemocrats             2
DemocratsAreDestroyingAmerica     2
DemocratsAreHypocrites            1
DemocratsAreDomesticTerrorists    1
dtype: int64

In [236]:
filter_top_hashtags(hashtag_counts, 'china', k = 20)

China         6
ChinaVirus    4
dtype: int64

In [210]:
find_tweet_from_hashtag(tweets, 'Democrats')

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quoted_status_permalink,quoted_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,extended_entities,hashtags
3654,Fri Oct 02 06:56:40 +0000 2020,1311923034064740353,1311923034064740353,#DonaldTrump contracts the #coronavirus • It a...,False,"[0, 234]","{'hashtags': [{'text': 'DonaldTrump', 'indices...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,NaN,NaN,0,1,False,False,NaN,en,NaN,"[DonaldTrump, coronavirus, Democrats, Repulica..."
3957,Fri Oct 02 15:28:37 +0000 2020,1312051869263433728,1312051869263433728,If Trump being of his age tests positive for #...,False,"[0, 287]","{'hashtags': [{'text': 'COVID19', 'indices': [...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,NaN,NaN,0,0,False,False,False,en,"{'media': [{'id': 1312051867052994560, 'id_str...","[COVID19, Democrats, Sad]"
4220,Fri Oct 02 13:30:18 +0000 2020,1312022094306316294,1312022094306316294,Now #Democrats are saying #Trump is faking hav...,False,"[0, 275]","{'hashtags': [{'text': 'Democrats', 'indices':...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,...,"{'url': 'https://t.co/RfmaX9zeeq', 'expanded':...",{'created_at': 'Fri Oct 02 10:39:44 +0000 2020...,0,0,False,False,False,en,NaN,"[Democrats, Trump, COVID19, coronavirus, Left,..."
5537,Sat Oct 03 01:39:44 +0000 2020,1312205661787152385,1312205661787152385,"On September 7, Trump asked @Reuters journalis...",False,"[0, 277]","{'hashtags': [{'text': 'COVID19', 'indices': [...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,...,NaN,NaN,1,2,False,False,False,en,"{'media': [{'id': 1303205514193428480, 'id_str...","[COVID19, Coronavirus, TrumpHasCovid, Democrat..."


### Scientific news

In [239]:
filter_top_hashtags(hashtag_counts, 'mask', k = 20)

WearAMask           32
WearADamnMask       20
MaskUp               6
mask                 5
Maskup               4
MaskUpAmerica        3
Masks                3
facemask             2
NoMasks              2
PutYourMaskOn        1
facemasks            1
Maskenpropaganda     1
ClothFaceMask        1
RespectTheMask       1
dtype: int64